In [2]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.4/1.4 MB 12.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [2]:
df = pd.read_csv('../rayundo2.csv', sep=';')

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (9,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
df.tail()

,Mês-ano,Categoria agrupada,Ano,Mês,Dia,RAZ_SOCIAL,nom_fantas,nom_fantas2,cliente,Column1,...,VLR_TOTAL,tipo_frete,Vendedor,ramo_ativ,Macroregião,Mesoregião,Depara.Tipo,ramo_ativ.1.DESCRICAO,Canal_novo,Canal_novo+velho
652418,1/5/2023,TIRA MANCHAS,2023,5,8,MASTER ATS SUPERMERCADOS LTDA,MASTER ATS,GRUPO REDE MASTER ATS,193619999.0,18741660014-14,...,1240,CIF,201,3.0,Região Nordeste,SÃO PAULO (REG. METROPOLITANA),VENDEDOR EXTERNO,SM 10 +,Cash & Carry Regional,Cash & Carry Regional
652419,1/5/2023,DESINFETANTE + MULTIUSO,2023,5,8,MASTER ATS SUPERMERCADOS LTDA,MASTER ATS,GRUPO REDE MASTER ATS,193619999.0,18741660014-14,...,621,CIF,201,3.0,Região Nordeste,SÃO PAULO (REG. METROPOLITANA),VENDEDOR EXTERNO,SM 10 +,Cash & Carry Regional,Cash & Carry Regional
652420,1/5/2023,DESINFETANTE + MULTIUSO,2023,5,8,MASTER ATS SUPERMERCADOS LTDA,MASTER ATS,GRUPO REDE MASTER ATS,193619999.0,18741660014-14,...,621,CIF,201,3.0,Região Nordeste,SÃO PAULO (REG. METROPOLITANA),VENDEDOR EXTERNO,SM 10 +,Cash & Carry Regional,Cash & Carry Regional
652421,1/5/2023,DESINFETANTE + MULTIUSO,2023,5,8,MASTER ATS SUPERMERCADOS LTDA,MASTER ATS,GRUPO REDE MASTER ATS,193619999.0,18741660014-14,...,"278,3",CIF,201,3.0,Região Nordeste,SÃO PAULO (REG. METROPOLITANA),VENDEDOR EXTERNO,SM 10 +,Cash & Carry Regional,Cash & Carry Regional
652422,1/5/2023,DETERGENTE LÍQUIDO,2023,5,8,COMERCIAL ESPERANCA ATACADO DISTRIB,ESPERANCA,NaN,NaN,490923980003-94,...,8662,CIF,201,4.0,Região Nordeste,SÃO PAULO (REG. METROPOLITANA),VENDEDOR EXTERNO,AT_CASH,Cash & Carry Regional,Cash & Carry Regional


In [8]:
df.columns

Index(['Mês-ano', 'Categoria agrupada', 'Ano', 'Mês', 'Dia', 'RAZ_SOCIAL',
       'nom_fantas', 'nom_fantas2', 'cliente', 'Column1', 'CIDADE', 'ESTADO',
       'Produto', '2', 'descricao', 'QTDE_Total', 'VLR_TOTAL', 'tipo_frete',
       'Vendedor', 'ramo_ativ', 'Macroregião', 'Mesoregião', 'Depara.Tipo',
       'ramo_ativ.1.DESCRICAO', 'Canal_novo', 'Canal_novo+velho'],
      dtype='object')

In [28]:
df['cliente'].value_counts()

In [5]:
df['grupo-rec'] = df['Mês-ano']+df['RAZ_SOCIAL']

In [6]:
def make_baskets(  df, group_column, product_column):

    basket = []
    for col in df[group_column].unique():
        df_tmp = df[ df[group_column] == col ]
        basket_items = df_tmp[product_column].dropna().unique().tolist()
        if len(basket_items) > 0:
            basket.append( basket_items )
    return basket

def encode_basket(basket):

    te = TransactionEncoder()
    te_ary = te.fit(basket).transform(basket)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    return df


In [7]:
basket_categoria = make_baskets(df, 'grupo-rec', 'Categoria agrupada')

In [43]:
basket_produto = make_baskets(df, 'grupo-rec', 'Produto')

In [57]:
print(basket_categoria[0:2])
print(basket_produto[0:2])

[['ÁGUA SANITÁRIA', 'ALVEJANTE', 'Amaciante', 'ÁLCOOL', 'DESINFETANTE + MULTIUSO', 'TIRA MANCHAS', 'LAVA ROUPAS LÍQUIDO'], ['ÁGUA SANITÁRIA', 'ALVEJANTE', 'Amaciante', 'DESINFETANTE + MULTIUSO', 'TIRA MANCHAS']]
[[1, 2, 3, 6, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 34, 39, 41, 42, 43, 44, 14, 52, 26], [1, 2, 6, 11, 12, 18, 19, 20, 53, 21]]


In [31]:
df_produto = encode_basket(basket_produto)

In [41]:
df_produto.head(2)

,1,2,3,6,7,11,12,13,14,15,...,87,88,89,90,91,92,93,94,95,97
0,True,True,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,True,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
df_categoria = encode_basket(basket_categoria)

In [59]:
frequent_produto = fpgrowth(df_produto, min_support=0.1, use_colnames=True)
rules_produto = association_rules(frequent_produto, metric="confidence", min_threshold=0.1)
#

In [9]:
frequent_categoria = fpgrowth(df_categoria, min_support=0.01, use_colnames=True)
rules_categoria = association_rules(frequent_categoria, metric="confidence", min_threshold=0.1)

In [62]:
rules_categoria

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(DESINFETANTE + MULTIUSO),(ÁGUA SANITÁRIA),0.800852,0.945754,0.769806,0.961234,1.016368,0.012397,1.399317,0.080866
1,(ÁGUA SANITÁRIA),(DESINFETANTE + MULTIUSO),0.945754,0.800852,0.769806,0.813960,1.016368,0.012397,1.070460,0.296874
2,(Amaciante),(ÁGUA SANITÁRIA),0.731129,0.945754,0.699623,0.956908,1.011794,0.008155,1.258850,0.043354
3,(ÁGUA SANITÁRIA),(Amaciante),0.945754,0.731129,0.699623,0.739752,1.011794,0.008155,1.033134,0.214884
4,(Amaciante),(DESINFETANTE + MULTIUSO),0.731129,0.800852,0.653529,0.893863,1.116140,0.068003,1.876333,0.387008
...,...,...,...,...,...,...,...,...,...,...
50655,"(DESINFETANTE + MULTIUSO, ÁGUA SANITÁRIA)","(Cloro Gel, TIRA MANCHAS)",0.769806,0.083637,0.078488,0.101959,1.219059,0.014104,1.020402,0.780625
50656,"(DESINFETANTE + MULTIUSO, TIRA MANCHAS)","(Cloro Gel, ÁGUA SANITÁRIA)",0.594134,0.099084,0.078488,0.132106,1.333274,0.019619,1.038048,0.615885
50657,"(ÁGUA SANITÁRIA, TIRA MANCHAS)","(Cloro Gel, DESINFETANTE + MULTIUSO)",0.662080,0.094762,0.078488,0.118548,1.251007,0.015748,1.026985,0.593762
50658,(Cloro Gel),"(DESINFETANTE + MULTIUSO, ÁGUA SANITÁRIA, TIRA...",0.101781,0.580864,0.078488,0.771153,1.327598,0.019368,1.831516,0.274721


In [61]:
frequent_categoria

,support,itemsets
0,0.945754,(ÁGUA SANITÁRIA)
1,0.800852,(DESINFETANTE + MULTIUSO)
2,0.731129,(Amaciante)
3,0.681541,(TIRA MANCHAS)
4,0.526863,(ALVEJANTE)
...,...,...
1675,0.017745,"(TIRA MANCHAS, Cloro Gel, DESINFETANTE + MULTI..."
1676,0.017438,"(ÁGUA SANITÁRIA, TIRA MANCHAS, Cloro Gel, DESI..."
1677,0.082442,"(Cloro Gel, ÁGUA SANITÁRIA, TIRA MANCHAS)"
1678,0.079439,"(Cloro Gel, DESINFETANTE + MULTIUSO, TIRA MANC..."


In [ ]:
grupos = frequent_grupo[ (frequent_grupo['length'] >= 2) & (frequent_grupo['support'] > 0.05)  ]

rec = {}
rec['recomendações'] = []
rec['codigo_cliente'] = []

for clientid in df['codigo_cliente'].unique():
    
    df_cliente = df[df['codigo_cliente'] == clientid]
    cliente_grupo = list(set(df_cliente['grupo_produto']))
    tmp_rec = []
    for item_comprado in cliente_grupo:
            for combination in grupos["itemsets"]:
                if item_comprado in combination:
                    for item in combination:
                        if item != item_comprado and item not in tmp_rec:
                            tmp_rec.append(item)

    rec3 = [item for item in tmp_rec if item not in cliente_grupo]
    rec['codigo_cliente'].append(clientid)
    rec['recomendações'].append(rec3)